# Inference time of Keras YOLO v2 models

This notebook calculates the average inference time per image for the Keras YOLO v2 models. This notebook should be run from inside the [root folder](https://github.com/experiencor/keras-yolo2) of the Keras YOLO git project. Also, the respective back-end, configuration file and trained model should also be placed in the same folder.

In [ ]:
import argparse
import os
import cv2
import numpy as np
from tqdm import tqdm
from preprocessing import parse_annotation
from utils import draw_boxes
from frontend import YOLO
import json
import time
import tensorflow as tf

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
# instructions from: https://michaelblogscode.wordpress.com/2017/10/10/reducing-and-profiling-gpu-memory-usage-in-keras-with-tensorflow-backend/

# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total of half the GPU memory to be allocated
#config.gpu_options.per_process_gpu_memory_fraction = 0.5
 
# Create a session with the above options specified.
tf.keras.backend.set_session(tf.Session(config=config))
#keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

Set/adjust the parameters required.

In [ ]:
config_path = './config_tiny_yolo.json'
weights_path = './tiny_yolo_singapore_dataset.h5'
input_folder = '/home/tbontz2s/git/tensorflow/workspace/training_demo/images/test'

In [ ]:
with open(config_path) as config_buffer:    
    config = json.load(config_buffer)

###############################
#   Make the model 
###############################

yolo = YOLO(backend             = config['model']['backend'],
            input_size          = config['model']['input_size'], 
            labels              = config['model']['labels'], 
            max_box_per_image   = config['model']['max_box_per_image'],
            anchors             = config['model']['anchors'])

###############################
#   Load trained weights
###############################    

yolo.load_weights(weights_path)

Find all the images to be used for testing.

In [ ]:
images = [os.path.join(input_folder, img) for img in sorted(os.listdir(input_folder))]


# select only the first 10 images for testing time
images= images[:10]

print(images[:3])

Usually the first image is inferred slower since it build the Network. For this reason it is excluded for correct results. This cell print both averages with first inference time included and excluded but the more correct version is the second.

In [7]:
times = np.zeros([len(images)])

for i, img in enumerate(images):
    image = cv2.imread(img)
    start1 = time.time()
    boxes = yolo.predict(image)
    end1 = time.time()
    #times.append(end1-start1)
    times[i] = end1 - start1
    #image = draw_boxes(image, boxes, config['model']['labels'])

    #print(len(boxes), 'boxes are found')    

    #cv2.imwrite(os.path.join(output_folder, img.split('/')[-1]), image)
    

# print average time based on the numpy array
print("Average time based on numpy array: ", np.mean(times))
print("Frames per second: ", 1 / np.mean(times))

# print average time based on the numpy array ignore first image that is slow
print("Average time based on numpy array: ", np.mean(times[1:]))
print("Frames per second: ", 1 / np.mean(times[1:]))

Average time in seconds per image:  0.1643756628036499
Average milliseconds per image:  164.4
Time:  1.643756628036499
Average time based on numpy array:  0.13391001224517823
Frames per second:  7.467701505165142
Average time based on numpy array:  0.02917223506503635
Frames per second:  34.279169826055764
